# DFAnalyzer Simple Example

This notebook will guide you to load a trace file generated by DFTracer and analyze the trace events using Dask.

In [1]:
%load_ext autoreload
%autoreload 2

## System imports for the notebook

In [2]:
import os
from pathlib import Path
import sys

## We add the analysis code to path so that we can run this in dev mode.

In [3]:
app_root = str(Path(os.getcwd()).parent.parent)
sys.path.insert(0, app_root)
app_root

'/usr/WS2/sinurat1/dftracer'

## Imports for the dfanalyzer

This may take some time as it initializes Dask and submitting jobs to Scheduler (e.g. Slurm)

In [4]:
from dftracer.analyzer import init_with_hydra

percentile = 0.9
time_granularity = 5  # 5 seconds
trace_path = f"/p/lustre3/sinurat1/results/ml_workloads/unet3d/2025-07-18-10-34-26-2742472-LLUR"
view_types = ["time_range", "proc_name"]

dfa = init_with_hydra(
    hydra_overrides=[
        "cluster=slurm",
        "cluster.cores=3",
        "cluster.memory=32GB",
        "cluster.job_directives_skip=['--mem']",
        f"cluster.log_directory={app_root}/examples/dfanalyzer/logs",
        "++cluster.n_workers=3",
        f"trace_path={trace_path}",
    ]
)

/usr/workspace/sinurat1/dftracer/.venv-corona/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44593 instead
  warnings.warn(


## Dask Client

This is needed for SSH forwarding in case we need to see the Dask Dashboard

In [5]:
dfa.client

<Client: 'tcp://192.168.128.82:46569' processes=3 threads=9, memory=89.40 GiB>

## Analyze the events

In [6]:
res = dfa.analyze_trace()

## Show the analysis

In [7]:
dfa.output.handle_result(res)

                                                Time Period Summary                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metric                                                         ┃ Unit                  ┃                  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Job Time                                                       │ seconds               │               1144.796 │
│ Total Count                                                    │ count                 │                 16,008 │
│ Total Files                                                    │ count                 │                    337 │
│ Total Nodes                                                    │ count                 │                      1 │
│ Total Processes                                                │ count                 │                      8 │
│ POSIX Count                                                    │ count                 │                 13,136 │
│ POSIX Size                                                     │ MB                    │               6400.000 │
│ POSIX Bandwidth                                                │ MB/s                  │               2539.527 │
│ POSIX Avg Transfer Size                                        │ MB                    │                  0.487 │
└────────────────────────────────────────────────────────────────┴───────────────────────┴────────────────────────┘
                                                  Layer Breakdown                                                  
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer      ┃        Time (s) ┃          Ops ┃         Ops/sec ┃         Size (MB) ┃            Bandwidth (MB/s) ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ POSIX      │           2.520 │       13,136 │        5212.380 │          6400.000 │                    2539.527 │
└────────────┴─────────────────┴──────────────┴─────────────────┴───────────────────┴─────────────────────────────┘